In [1]:
import awkward as awk
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd
from scipy import stats
from scipy.optimize import brentq
from scipy.interpolate import griddata
import os
import seaborn as sns

sns.set_style('whitegrid')

n_m = 10
n_mu = 10
m_sample = np.geomspace(1e-2,2,n_m)
mu_sample = np.geomspace(1e-7,1e-5,n_mu)
m_sample, mu_sample = np.meshgrid(m_sample, mu_sample)
m_sample = np.reshape(m_sample,[n_m*n_mu])
mu_sample = np.reshape(mu_sample,[n_m*n_mu])

plt.rcParams.update({
    "text.usetex": False,
    "font.family": "sans-serif",
    "font.sans-serif": ["Arial", "DejaVu Sans", "Helvetica", "sans-serif"],
    "mathtext.fontset": "dejavusans",
    'font.size': 16,
    'image.cmap': 'YlGnBu',
})

POT = {}
bkg = {}

# POT used in MC simulation
POT['ND280+_MC'] = 2.6098758621e22
POT['ND280_MC'] = 3.6e21

# POT in Abe et al 2019
POT['ND280_nu_2019'] = 1.234e21
POT['ND280_nubar_2019'] = 6.29e20

POT['ND280_nu_2023'] = 19.7e20
POT['ND280_nubar_2023'] = 16.3e20

# POT Events conducted in ND280+
# 1E22 is cumulative
POT['ND280+_nu'] = 1e22 - POT['ND280_nu_2023'] 
POT['ND280+_nubar'] = 1e22 - POT['ND280_nubar_2023']


# TPC e+e- background from Abe et al 2019
bkg['ND280_nu_2019'] = 0.563
bkg['ND280_nubar_2019'] = 0.015 

bkg['ND280+_nu'] = bkg['ND280_nu_2019'] * 5 / 3 * POT['ND280+_nu'] / POT['ND280_nu_2019'] 
bkg['ND280+_nubar'] = bkg['ND280_nubar_2019']* 5 / 3 * POT['ND280+_nubar'] / POT['ND280_nu_2019'] 

bkg['ND280_nu_2023'] = bkg['ND280_nu_2019'] * 5 / 3 * POT['ND280_nu_2023'] / POT['ND280_nu_2019'] 
bkg['ND280_nubar_2023'] = bkg['ND280_nubar_2019']* 5 / 3 * POT['ND280_nubar_2023'] / POT['ND280_nu_2019'] 

In [15]:
m_sample = m_sample[60:70]
mu_sample = mu_sample[60:70]

In [ ]:
POT_weight = POT['ND280+_nu']/POT['ND280+_MC']
POT_weight_OG = POT['ND280_nu_2023']/POT['ND280_MC']


def BR(m):
    alpha = 1/137 # Fine structure constant
    me = 0.511e-3 # Electron mass in GeV
    r = me/m
    L = lambda r:(2 - r**6 * 0.125) * np.arccosh(1/r) - (24 - 10 * r**2 + r**4) * 0.125 * np.sqrt(1 - 4*r**2)
    return np.array(alpha * L(r) / (3 * np.pi))

def get_data_csv(scale='log', year=3, exp='ND280+', mode = 'nu'):
    
    '''
    Scale = ['log' or 'linear']
    Year = 0, 1, 2, 3 only applies to ND280+ projection
    Mode = ['nu', 'nubar']
    '''
    
    if exp == 'ND280+' and mode == 'nu':
        path='../stats/signal_strengths.csv'
    elif exp == 'ND280' and mode == 'nu':
        path='../stats/signal_strengths_nd280.csv'
    elif exp == 'ND280+' and mode == 'nubar':
        path='../stats/signal_strengths_nubar.csv'
    elif exp == 'ND280' and mode == 'nubar':
        path='../stats/signal_strengths_nd280_nubar.csv'    
        
    df = pd.read_csv(path).sort_values(['Experiment', 'i'])
    grouped = df.groupby('Experiment')
    
    signal_strengths = {exp: group.drop(['Experiment', 'i'], axis=1).to_numpy() 
                        for exp, group in grouped}
    
    last_exp = list(signal_strengths.keys())[-1]
    mass = signal_strengths[last_exp][:, 0]
    coupling = signal_strengths[last_exp][:, 1]
    
    total_signal_strengths = np.sum([arr[:, 2] for arr in signal_strengths.values()], axis=0)
    
    # Downweight by branching ratio
    total_signal_strengths *= BR(mass)
    
    # Scale by respective POT
    if exp == 'ND280+':
        if mode =='nu':
            ratio = POT['ND280+_nu']/POT['ND280+_MC'] * year / 3 
        else:
            ratio = POT['ND280+_nubar']/POT['ND280+_MC'] * year / 3

    elif exp == 'ND280':
        if mode =='nu':
            ratio = POT['ND280_nu_2019']/POT['ND280_MC']
        else:
            ratio = POT['ND280_nubar_2019']/POT['ND280_MC']
            
    total_signal_strengths *= ratio
    
    # COUPLING/2 = TMM 
    if scale == 'log':
        return (np.log10(mass * 1000), np.log10(coupling / 2), np.log10(total_signal_strengths))
    return (np.log10(mass * 1000), np.log10(coupling / 2), total_signal_strengths)

signal_threshold = {}

def logLR(z, key='ND280+_nu'):
    return np.absolute(z - (bkg[key]+z) * np.log(1 + z/bkg[key]))

# 95% Confidence Level
def tar(cl):
    return 0.5 * stats.norm.ppf(1-cl)**2